In [1]:
import pandas as pd
import numpy as np

df2=pd.read_excel('/content/drive/MyDrive/maszynowe16.xlsx', index_col=False)
df2=df2[df2['kapitalizacja'] != 0]
df2['wzrost'] = np.where(df2['symbol'].shift(-1) == df2['symbol'], (df2['kapitalizacja'].shift(-1)-df2['kapitalizacja'])/df2['kapitalizacja'].shift(-1), 0)
df2['wzrost2'] = np.where(df2['wzrost'] > 0.05,1,0)
df2['wzrost3'] = np.where(df2['wzrost'] < -0.10,1,0)
df2['wzrost'] = np.where(df2['wzrost'] > 0.19,1,0)
print(df2)


       Unnamed: 0 symbol           okres Płynność bieżąca  \
0             275    PCO  2021/Q2(mar21)     (0.96, 1.18]   
1             276    PCO             AKT     (0.96, 1.18]   
2             615    GOV  2017/Q4(gru17)     (3.72, 7.29]   
3             616    GOV  2018/Q1(mar18)     (3.72, 7.29]   
4             617    GOV  2018/Q2(cze18)     (3.72, 7.29]   
...           ...    ...             ...              ...   
36894       16012    BST  2024/Q1(mar24)     (3.72, 7.29]   
36895       16014    BST  2024/Q2(cze24)     (3.72, 7.29]   
36896       39697    BST  2024/Q3(wrz24)     (3.72, 7.29]   
36897      126333    BST  2024/Q4(gru24)     (3.72, 7.29]   
36898      126334    BST             AKT     (3.72, 7.29]   

      Płynność bieżącaproc Płynność bieżącaprocROK Płynność bieżącaprocROK2  \
0           (-0.0319, 0.0]          (-0.0473, 0.0]           (-0.0305, 0.0]   
1           (-0.0319, 0.0]          (-0.0473, 0.0]           (-0.0305, 0.0]   
2           (-0.0319, 0.0]    

In [2]:
df=df2.drop(columns=['okres','symbol','Unnamed: 0','kapitalizacja','wzrost2','wzrost3'])
for col in df.columns:

  #print(x[col].value_counts())
  df=pd.get_dummies(df, columns=[col],drop_first=True)
print(df.shape)



x= df.drop(columns=['wzrost_1'])
x=x.loc[~(x==0).all(axis=1)]
#print(x)

(36883, 414)


In [3]:
from keras.models import load_model

model = load_model('/content/drive/MyDrive/moj_najlepszy_model_WK_spadek_03_02_2025.keras')

model.summary()

wagilist=model.get_weights()

print(wagilist[0])

# dfWagi=pd.DataFrame(wagilist[0])
# dfWagi.to_excel('/content/drive/MyDrive/wagi.xlsx')

# lll=dfAKT.drop(columns=['wzrost_1'])
# lll=lll.columns
# lll=pd.DataFrame(lll)
# lll.to_excel('/content/drive/MyDrive/kolumny.xlsx')
# print(lll)


prediction = model.predict(df.drop(columns=['wzrost_1']))
#prediction = model.predict(df.drop(columns=['wzrost_1']))
#prediction = model.predict(x_test)

symbolList=df2['symbol'].to_list()
okresList=df2['okres'].to_list()
listaIDX=df2.index.to_list()
listaIDX=df2.index.to_list()
c=0
ilosc_prawd=0
ilosc_prawd_5_proc=0
minus_10_procc=0
www=0
ggg=0

### raport ###

DFraport=pd.DataFrame(df2['symbol'].unique() )
DFraport=DFraport.set_index(0)

for x in prediction:
  symbol=df2.loc[listaIDX[c],'symbol']
  if df2.loc[listaIDX[c],'okres'].find('wrz24') !=-1 or df2.loc[listaIDX[c],'okres'].find('paź24') !=-1:

      DFraport.loc[symbol,'symbol']=df2.loc[listaIDX[c],'symbol']
      DFraport.loc[symbol,'okresPoprzedni']= df2.loc[listaIDX[c],'okres']
      DFraport.loc[symbol,'poprzedniKwartalSpadek']=  round(x[0],2)
  if df2.loc[listaIDX[c],'okres'].find('gru24') !=-1 or df2.loc[listaIDX[c],'okres'].find('sty25') !=-1:
      DFraport.loc[symbol,'symbol']=df2.loc[listaIDX[c],'symbol']
      DFraport.loc[symbol,'okresAktualny']= df2.loc[listaIDX[c],'okres']
      DFraport.loc[symbol,'AktualnyKwartalSpadek']=  round(x[0],2)
  if df2.loc[listaIDX[c],'okres'].find('AKT') !=-1:
      DFraport.loc[symbol,'symbol']=df2.loc[listaIDX[c],'symbol']
      DFraport.loc[symbol,'okresAktualnyCENA']= df2.loc[listaIDX[c],'okres']
      DFraport.loc[symbol,'AktualnyKwartalSpadekCENA']=  round(x[0],2)
  #if str(round(x[0],2)) == '0.12':
  if x[0] > 0.4:
    www=www+1
    if str(df2.loc[listaIDX[c],'wzrost']) == '1':
      ilosc_prawd=ilosc_prawd+1
    if str(df2.loc[listaIDX[c],'wzrost2']) == '1':
      ilosc_prawd_5_proc=ilosc_prawd_5_proc+1
    if str(df2.loc[listaIDX[c],'wzrost3']) == '1':
      minus_10_procc=minus_10_procc+1
  if str(df2.loc[listaIDX[c],'wzrost']) == '1':
    ggg=ggg+1
  c=c+1





print(DFraport)
DFraport.to_excel('/content/drive/MyDrive/raportSpadek.xlsx')
print(ilosc_prawd)
print(www)
print('procent + 20%')
print((ilosc_prawd/www)*100)
print('procent + 5%')
print((ilosc_prawd_5_proc/www)*100)
print('procent <> -10%')
print((minus_10_procc/www)*100)
print('procent + 20% w test')
print((ilosc_prawd/ggg)*100)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                     │ (None, 512)                 │         211,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 637,445 (2.43 MB)

 Trainable params: 212,481 (830.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 424,964 (1.62 MB)

[[-0.05329704  0.04061877 -0.00655272 ... -0.06718449  0.11859459
   0.0192037 ]
 [-0.04412978 -0.13024366  0.06759124 ... -0.07326037 -0.04323988
   0.04299246]
 [ 0.02363718 -0.00521315 -0.0445457  ...  0.02270664 -0.0711261
  -0.08769112]
 ...
 [ 0.05092398 -0.00030396  0.0587859  ...  0.1043551  -0.08191348
  -0.06569224]
 [ 0.0325506  -0.17899942  0.00775971 ... -0.04569896 -0.00380472
   0.02679832]
 [-0.03196958  0.07518066  0.08933634 ... -0.13667253  0.0439951
   0.04197967]]
1153/1153 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
    symbol okresAktualnyCENA  AktualnyKwartalSpadekCENA  okresPoprzedni  \
0                                                                         
PCO    PCO               AKT                       0.31             NaN   
GOV    GOV               AKT                       0.18             NaN   
MGT    MGT               AKT                       0.13  2024/Q3(wrz24)   
ACG    ACG               AKT                       0.08  2024/Q3(wrz24)   
ATS    ATS       

In [4]:
from keras.models import load_model

model = load_model('/content/drive/MyDrive/moj_najlepszy_model_wzrost_WK_03_02_2025.keras')

model.summary()

wagilist=model.get_weights()

print(wagilist[0])

dfWagi=pd.DataFrame(wagilist[0])
dfWagi['srednia']=dfWagi.mean(axis=1)
dfWagi=dfWagi.sort_values(by='srednia',ascending=False)
#dfWagi.to_excel('/content/drive/MyDrive/wagiMAKRO_EKO.xlsx')

lll=df.drop(columns=['wzrost_1'])
lll=lll.columns
lll=pd.DataFrame(lll)
#lll.to_excel('/content/drive/MyDrive/kolumnyMAKRO_EKO.xlsx')
print(lll)

dfWagi['kolumny']=lll
#dfWagi.to_excel('/content/drive/MyDrive/wagiMAKRO_EKO.xlsx')
lll=df.drop(columns=['wzrost_1'])
lll=lll.columns
lll=pd.DataFrame(lll)
#lll.to_excel('/content/drive/MyDrive/kolumnyMAKRO_EKO.xlsx')
print(lll)

dfWagi['kolumny']=lll
dfWagi=dfWagi[['kolumny','srednia']]
dfWagi.to_excel('/content/drive/MyDrive/wagiMAKRO_EKO.xlsx')


prediction = model.predict(df.drop(columns=['wzrost_1']))
#prediction = model.predict(df.drop(columns=['wzrost_1']))
#prediction = model.predict(x_test)

symbolList=df2['symbol'].to_list()
okresList=df2['okres'].to_list()
listaIDX=df2.index.to_list()
listaIDX=df2.index.to_list()
c=0
ilosc_prawd=0
ilosc_prawd_5_proc=0
minus_10_procc=0
www=0
ggg=0

### raport ###


for x in prediction:
  symbol=df2.loc[listaIDX[c],'symbol']
  if df2.loc[listaIDX[c],'okres'].find('wrz24') !=-1 or df2.loc[listaIDX[c],'okres'].find('paź24') !=-1:

      DFraport.loc[symbol,'symbol']=df2.loc[listaIDX[c],'symbol']
      DFraport.loc[symbol,'okresPoprzedni']= df2.loc[listaIDX[c],'okres']
      DFraport.loc[symbol,'poprzedniKwartalWzrost']=  round(x[0],2)
  if df2.loc[listaIDX[c],'okres'].find('gru24') !=-1 or df2.loc[listaIDX[c],'okres'].find('sty25') !=-1:
      DFraport.loc[symbol,'symbol']=df2.loc[listaIDX[c],'symbol']
      DFraport.loc[symbol,'okresAktualny']= df2.loc[listaIDX[c],'okres']
      DFraport.loc[symbol,'AktualnyKwartalWzrost']=  round(x[0],2)

  if df2.loc[listaIDX[c],'okres'].find('AKT') !=-1:
      DFraport.loc[symbol,'symbol']=df2.loc[listaIDX[c],'symbol']
      DFraport.loc[symbol,'okresAktualnyCENA']= df2.loc[listaIDX[c],'okres']
      DFraport.loc[symbol,'AktualnyKwartalWzrostCENA']=  round(x[0],2)
  #if str(round(x[0],2)) == '0.12':
  if x[0] > 0.4:
    if (df2.loc[listaIDX[c],'okres'].find('wrz24') !=-1 or df2.loc[listaIDX[c],'okres'].find('paź24') !=-1):
      print(df2.loc[listaIDX[c],'symbol'])
      print(df2.loc[listaIDX[c],'okres'])
      print(round(x[0],2))
    www=www+1
    if str(df2.loc[listaIDX[c],'wzrost']) == '1':
      ilosc_prawd=ilosc_prawd+1
    if str(df2.loc[listaIDX[c],'wzrost2']) == '1':
      ilosc_prawd_5_proc=ilosc_prawd_5_proc+1
    if str(df2.loc[listaIDX[c],'wzrost3']) == '1':
      minus_10_procc=minus_10_procc+1
  if str(df2.loc[listaIDX[c],'wzrost']) == '1':
    ggg=ggg+1
  c=c+1





print(DFraport)
DFraport.to_excel('/content/drive/MyDrive/raportSpadek.xlsx')
print(ilosc_prawd)
print(www)
print('procent + 20%')
print((ilosc_prawd/www)*100)
print('procent + 5%')
print((ilosc_prawd_5_proc/www)*100)
print('procent <> -10%')
print((minus_10_procc/www)*100)
print('procent + 20% w test')
print((ilosc_prawd/ggg)*100)

Model: "sequential_51"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_102 (Dense)                    │ (None, 2048)                │         847,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_51 (Dropout)                 │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_103 (Dense)                    │ (None, 1)                   │           2,049 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,549,765 (9.73 MB)

 Trainable params: 849,921 (3.24 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,699,844 (6.48 MB)

[[-0.03787415  0.04758457  0.04652132 ...  0.02754699 -0.00379598
   0.01021482]
 [ 0.02347274  0.00906931 -0.05055351 ...  0.03148712  0.02394423
  -0.04361128]
 [-0.00145791  0.02875619  0.00368124 ...  0.04417746 -0.04173401
  -0.02269841]
 ...
 [-0.03907822 -0.0099119   0.01906909 ... -0.02862317  0.01257079
  -0.01719283]
 [ 0.01240403 -0.05468609  0.03653768 ...  0.0334177   0.01395216
  -0.04721389]
 [ 0.04094205  0.00725476 -0.05240742 ... -0.0325441  -0.00492632
  -0.005977  ]]
                                                    0
0                       Płynność bieżąca_(0.57, 0.96]
1                       Płynność bieżąca_(0.96, 1.18]
2                       Płynność bieżąca_(1.18, 1.38]
3                       Płynność bieżąca_(1.38, 1.62]
4                       Płynność bieżąca_(1.62, 1.97]
..                                                ...
408  Enterprise Value na akcjęprocROK2_(-inf, -1.375]
409    Enterprise Value na akcjęprocROK2_(0.0, 0.112]
410  Enterprise Value 